**Simple TEXT Classification using TF-IDF features**

In [0]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
import pandas, xgboost, numpy, textblob, string

In [22]:
# load the dataset
data = open('/corpus').read()
labels, texts = [], []
#flag=0
for i, line in enumerate(data.split("\n")):                                      # read the data load it and segregate w.r.t space and divide & assign the content array into two called labels and text.
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))
    if(i<7):                                                                     # to view the corpus file data before dataframing sample 7
      print('Label:', labels[i],"\n",'Text:', content[1:])
       
'''
    if(flag==0):
      print('Label:', labels[i],"\n",'Text:', content[1:])
      flag=1 
'''  

# create a dataframe using texts and lables
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

print ("\n",trainDF)                                                             #complete dataframe every row and column matrix format
print("\n",trainDF.iloc[3])                                                      #row vise selection in dataframe

Label: __label__2 
 Text: ['Stuning', 'even', 'for', 'the', 'non-gamer:', 'This', 'sound', 'track', 'was', 'beautiful!', 'It', 'paints', 'the', 'senery', 'in', 'your', 'mind', 'so', 'well', 'I', 'would', 'recomend', 'it', 'even', 'to', 'people', 'who', 'hate', 'vid.', 'game', 'music!', 'I', 'have', 'played', 'the', 'game', 'Chrono', 'Cross', 'but', 'out', 'of', 'all', 'of', 'the', 'games', 'I', 'have', 'ever', 'played', 'it', 'has', 'the', 'best', 'music!', 'It', 'backs', 'away', 'from', 'crude', 'keyboarding', 'and', 'takes', 'a', 'fresher', 'step', 'with', 'grate', 'guitars', 'and', 'soulful', 'orchestras.', 'It', 'would', 'impress', 'anyone', 'who', 'cares', 'to', 'listen!', '^_^']
Label: __label__2 
 Text: ['The', 'best', 'soundtrack', 'ever', 'to', 'anything.:', "I'm", 'reading', 'a', 'lot', 'of', 'reviews', 'saying', 'that', 'this', 'is', 'the', 'best', "'game", "soundtrack'", 'and', 'I', 'figured', 'that', "I'd", 'write', 'a', 'review', 'to', 'disagree', 'a', 'bit.', 'This', 'in

In [23]:
# split the dataset into training and validation datasets randomly >>> x represents text dataframe and y => label dataframe 
#train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])
#train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.40, random_state=32)
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.25, random_state=32, stratify=labels)



print("Training set of text train_x""\n")
print(train_x.iloc[3])                                                           # to check training set of text

print("\n""respective training set of label train_y""\n")
print(train_y.iloc[3])                                                           # to check the training set of label

# label encode the target variable i.e convert the labels into 0 to n-class-1
encoder = preprocessing.LabelEncoder()                                           # 'encoder' is the object
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#encoded results of 7 labels
print(train_y[0:7])


#In order to reverse the encoded format of the train_y and valid_y

#list(encoder.inverse_transform(train_y))



Training set of text train_x

This ain't no economics book!: I was expecting a book on economics 101, so maybe I might understand market forces, PE ratios, stuff like that.It's just a big argument for a libertarian government. Not even particularly good arguments either - they're all based on metaphors, like the broken window (paraphrased from memory):'Even though they may get paid, a community does not get rich fixing broken windows. That holds the community still, it does not push it forward.'That's a fine argument and all, but that's not teaching. Metaphors are mental manipulation... give me all the pros and cons and let me decide. Or better: give me the damn facts.And it's outdated too. The book is 50 years old. We are so NOT in Post WWII economics.

respective training set of label train_y

__label__1
[0 1 1 0 1 1 0]


**STEP 2**: ***Feature Selection***

 ***Tf-Idf features***
 
  **a) word level tf-idf features**

In [9]:
# word level tf-idf

tfidf_vector = TfidfVectorizer(analyzer='word', token_pattern=r'\w+', max_features=7000)
#tfidf_vector = TfidfVectorizer(analyzer='word', token_pattern=r'\w+')

tfidf_vector.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vector.transform(train_x)
xvalid_tfidf =  tfidf_vector.transform(valid_x)

print(xtrain_tfidf.shape)
print(xvalid_tfidf.shape)


(6000, 7000)
(4000, 7000)


**b) n-gram tf-idf features**

In [27]:
# ngram level tf-idf 
#tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w+', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3))  # accuracy= 49.925, 60:40, stratify

tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

print(xtrain_tfidf_ngram.shape)                                      #(7500, 870536) matrix shape without maxfeature
print(xvalid_tfidf_ngram.shape)                                      #(2500, 870536)

(7500, 870536)
(2500, 870536)


**STEP 3: MODEL BUILDING and ACCURACY FINDING**


In [28]:
# function to train the model
def train_test_model(classifier, feature_vector_train, label, feature_vector_valid, name):
                                                                                 # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
                                                                                 # save the trained model in the ".pkl" file so that objects are serialised into the pickle file.
    joblib.dump(classifier, '/' + name + '.pkl') 

                                                                                 # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    return metrics.accuracy_score(predictions, valid_y)


# Naive Bayes on Count Vectors
accuracy = train_test_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram,'NB-Count')
print("NB, TFIDF Vectors: ", accuracy*100)


NB, TFIDF Vectors:  84.88


# **OUTPUT REVIEWS:**

**Dataset: Amazon review dataset (10000 reviews, 2 label)**

**word level**
**features: 31,666 (without threshold "maxfeatures")**
**accuracy: 81.35**

**features: 7000 (user given maxfeature setting)**
**accuracy: 83.375**

**n-gram word level**
**features: 870536 (without threshold "maxfeatures") 70:30**
**accuracy: 81.35** 